In [61]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf
import os
import cv2

import keras

from sklearn.metrics import confusion_matrix, classification_report 




In [62]:
image_dir=Path('preprocessed')


In [63]:
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df1 = pd.concat([filepaths, labels], axis=1)


In [64]:

# # Import label encoder
# from sklearn import preprocessing
  
# # label_encoder object knows how to understand word labels.
# label_encoder = preprocessing.LabelEncoder()
  
# # Encode labels in column 'species'.
# image_df1['Label']= label_encoder.fit_transform(image_df1['Label'])
  
# image_df1['Label'].unique()

In [65]:
image_df1

,Filepath,Label
0,preprocessed/21_gaf/gaf_mid/20.png,gaf_mid
1,preprocessed/21_gaf/gaf_mid/22.png,gaf_mid
2,preprocessed/21_gaf/gaf_mid/42.png,gaf_mid
3,preprocessed/21_gaf/gaf_mid/49.png,gaf_mid
4,preprocessed/21_gaf/gaf_mid/21.png,gaf_mid
...,...,...
835,preprocessed/25_non/non_begin/66.png,non_begin
836,preprocessed/25_non/non_begin/327.png,non_begin
837,preprocessed/25_non/non_begin/387.png,non_begin
838,preprocessed/25_non/non_begin/185.png,non_begin


In [66]:
image_df1.to_csv('image_labels.csv',index=False) 

In [67]:
train_df, test_df = train_test_split(image_df1, train_size=0.7, shuffle=True, random_state=7)

In [68]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [69]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(10, 22),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=20,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(10, 22),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=20,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(10, 22),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=20,
    shuffle=False
)


Found 471 validated image filenames belonging to 42 classes.
Found 117 validated image filenames belonging to 42 classes.
Found 252 validated image filenames belonging to 42 classes.


In [70]:

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras import layers
from keras.utils import to_categorical, image_dataset_from_directory
from sklearn.metrics import confusion_matrix

model = tf.keras.models.Sequential()
inputs = tf.keras.Input(shape=(10, 22))
# Preprocessing 
model.add(layers.Resizing(height=128, width=128, interpolation='bilinear'))
model.add(layers.Rescaling(scale=1./255))

# Data Augmentation
#model.add(layers.RandomFlip(mode='horizontal_and_vertical'))
#model.add(layers.RandomRotation(factor=.2))

# First Convolution
model.add(layers.Conv2D(filters=64, kernel_size=(1,5), input_shape = (224,224,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(5,1)))
model.add(layers.Activation('leaky_relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(.2))

# Second Convolution
model.add(layers.Conv2D(filters=64, kernel_size=(1,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,1)))
model.add(layers.Activation('leaky_relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(.2))

# Third Convolution
model.add(layers.Conv2D(filters=64, kernel_size=(1,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(3,1)))
model.add(layers.Activation('leaky_relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Dropout(.2))

# Feed forward
model.add(layers.Flatten())
model.add(layers.Dense(512))
model.add(layers.Activation('leaky_relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(512))
model.add(layers.Activation('leaky_relu'))
model.add(layers.Dropout(.2))
model.add(layers.Dense(42))
model.add(layers.Activation('softmax'))


In [72]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
         tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            patience=3
        )
    ]
)

Epoch 1/50


2023-05-02 06:52:44.331601: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


24/24 [==============================] - ETA: 0s - loss: 6.0225 - accuracy: 0.0446

2023-05-02 06:52:53.921268: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


24/24 [==============================] - 10s 376ms/step - loss: 6.0225 - accuracy: 0.0446 - val_loss: 10.1135 - val_accuracy: 0.0256 - lr: 0.0010
Epoch 2/50
24/24 [==============================] - 9s 376ms/step - loss: 5.2583 - accuracy: 0.0340 - val_loss: 16.3112 - val_accuracy: 0.0256 - lr: 0.0010
Epoch 3/50
24/24 [==============================] - 9s 374ms/step - loss: 4.4242 - accuracy: 0.0318 - val_loss: 20.5185 - val_accuracy: 0.0342 - lr: 0.0010
Epoch 4/50
24/24 [==============================] - 9s 379ms/step - loss: 4.1696 - accuracy: 0.0616 - val_loss: 24.2464 - val_accuracy: 0.0342 - lr: 0.0010
Epoch 5/50
24/24 [==============================] - 9s 382ms/step - loss: 3.8482 - accuracy: 0.0701 - val_loss: 20.8221 - val_accuracy: 0.0342 - lr: 1.0000e-04
Epoch 6/50
24/24 [==============================] - 9s 377ms/step - loss: 3.3389 - accuracy: 0.0977 - val_loss: 20.9133 - val_accuracy: 0.0342 - lr: 1.0000e-04


In [ ]:
model.save('Model.h5')


In [73]:
from keras.models import load_model

# Load the saved model
model = load_model('Model.h5')

# Use the model to make predictions on new data
predictions = model.predict(train_images)

ValueError: The channel dimension of the inputs should be defined. The input_shape received is (None, 128, 128, None), where axis -1 (0-based) is the channel dimension, which found to be `None`.